<a href="https://colab.research.google.com/github/jcalandra/audiosynthesis_dl/blob/master/src/dataset_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2, numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files

In [3]:
! git clone https://github.com/jcalandra/audiosynthesis_dl.git

Cloning into 'audiosynthesis_dl'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (261/261), done.
remote: Total 613 (delta 13), reused 259 (delta 8), pack-reused 339
Receiving objects: 100% (613/613), 9.71 MiB | 42.15 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [0]:
#GLOBAL VALUES

NB_CLASS = 12
PICT_WIDTH = 400
NB_CHANNEL = 3
LINE_WIDTH = PICT_WIDTH//NB_CLASS #33

# number of trained and validation pictures based on MNIST dataset
NB_TRAIN = 600
NB_TEST = 100


In [0]:
#LOADING THE BASELINE

baseline = cv2.imread('audiosynthesis_dl/data/base_quadrillage.png', 0)
#cv2_imshow(baseline)

In [0]:
#PICTURE GENERATION

def generate_pict(nb_pict, folder):
  """ creates nb_pictures of pitch-lines and saves them permanently in the folder img_'folder'.
      Folder has to be a string and nb_pict is an integer."""
  for i in range (nb_pict) :
    # creation of the baseline : quadrilled picture
    line_image = np.zeros((PICT_WIDTH,PICT_WIDTH,NB_CHANNEL), np.uint8)
    line_image = baseline.copy()

    # generation of lines :
    thickness = np.random.randint((LINE_WIDTH - 4)//2) # maximum of (LINE_WIDTH - 4) to obtain in this case a bit of variation in the shape and avoid the creation of two exact same pictures
    delta = (LINE_WIDTH - (thickness*2))//2            # delta of allowed variation of the line
    height = (i%NB_CLASS)*LINE_WIDTH + (LINE_WIDTH + 5)//2   # actual height depending on (i%12) the pitch and the width of a pitch line

    variation = 0
    for j in  range (PICT_WIDTH):
      intervalle = np.random.randint(1,12)              #to avoid a sharp variation, we keep the same variation height for each 'intervalle'
      if (j%intervalle == 0) :                         # if we want to change the height of the line
        tmp_var = np.random.randint(-1, 2)             # each variation is an increase or a decrease of 1 (or same height)
        if abs(variation + tmp_var) < delta :
          variation = variation + tmp_var
        else :
          variation = variation

      line_image[PICT_WIDTH - height - 1 + variation][PICT_WIDTH - j - 1] = 0       # creation of the line
      for k in range (thickness) :
        line_image[PICT_WIDTH - height - 1 - k + variation][PICT_WIDTH - j - 1] = 0 # and its thickness
        line_image[PICT_WIDTH - height - 1 + k + variation][PICT_WIDTH - j - 1] = 0

    #cv2_imshow(line_image) #showing the images for tests

    # save the picture in google colab :
    cv2.imwrite('./audiosynthesis_dl/data/pitch_img/img_'+folder+'/draw'+str(69 + i%NB_CLASS)+'_'+str(i//NB_CLASS)+'_'+folder+'.png', line_image) #69 is pitch for la440
  
generate_pict(NB_TRAIN, 'train')
generate_pict(NB_TEST, 'test')


In [0]:
#SAVING PICTURES
# save the pictures in local :
# - Think about changing dowload path in google chrome parameters according to the corresponding folder
# - Put the extension _test.png ot _train.png according to the corresponding folder
for i in range (NB_TRAIN):
  files.download('./audiosynthesis_dl/data/pitch_img/img_train/draw'+str(69 + i%12)+'_'+str(i//12)+'_train.png')
# for i in range (NB_TEST):
#   files.download('./audiosynthesis_dl/data/pitch_img/img_test/draw'+str(69 + i%12)+'_'+str(i//12)+'_test.png')
